In [1]:
# --- CELDA 2: Importaciones ---
import os

In [2]:
# --- CELDA 3: API Key ---
# Usa la API key de Groq (ó podés usar OpenAI cambiando el LLM)

os.environ["GROQ_API_KEY"] = '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'

In [ ]:
# --- CELDA 4: Bajar un PDF de ejemplo para analizar ---
# Puedes cambiar esta URL por cualquier PDF público que quieras probar
import requests

pdf_url = "https://www.oas.org/juridico/PDFs/mesicic3_arg_ley21526.pdf"  # Ley argentina, pública
pdf_fn = "ley_entidades_financieras.pdf"

r = requests.get(pdf_url)
with open(pdf_fn, "wb") as f:
    f.write(r.content)

print("PDF descargado:", pdf_fn)

PDF descargado: ley_entidades_financieras.pdf


# Demo: Chat con tus propios datos usando RAG + LlamaIndex + Groq

**¿Qué vas a aprender?**

1. Cargar un PDF y convertirlo a textos/chunks.
2. Crear un índice de búsqueda semántica sobre el PDF.
3. Conectar un LLM potente (Llama 3 via Groq).
4. Consultar en lenguaje natural cualquier cosa sobre el PDF (¡en español!).
5. Ver ejemplos y estrategias de prompt.


In [6]:
# --- CELDA 6: Configuración principal de LlamaIndex y Groq ---
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PDFReader
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Configuramos el LLM
llm = Groq(model="llama3-8b-8192", api_key=userdata.get('GROQ_API_KEY'))
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("LLM y embeddings configurados")

c:\Users\valec\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# --- CELDA 7: Leer e indexar el PDF ---
# Leemos el PDF (puede tardar si es muy largo)
loader = PDFReader()
documents = loader.load_data(file=pdf_fn)

# Mostramos una muestra de lo cargado:
print("Muestra de texto del PDF:\n")
print(documents[0].text[:800]) # Primeros 800 caracteres

# Creamos el índice vectorial para búsquedas semánticas
index = VectorStoreIndex.from_documents(documents)
print("Índice construido.")

Muestra de texto del PDF:

Ley 21.526 
LEY 21.526 
Buenos Aires, 14 de febrero de 1977 
B.O.: 21/2/77 
Entidades financieras. Con las modificaciones introducidas por Ley 24.144 
(B.O.: 22/10/92), Dto. 146/94 (B.O.: 21/2/94), Leyes 24.485 (B.O.: 
29/9/95), y 24.627 (B.O.: 18/3/96), Dto. 214/02 (B.O.: 4/2/02), y Leyes 
25.562 (B.O.: 8/2/02), 25.780 (B.O.: 8/9/03), 25.782 (B.O.: 31/10/03) y 
26.173 (B.O.: 12/12/06). 
* Títulos de los artículos incluidos por la Editorial. 
 
TITULO I - Régimen general 
CAPITULO I - Ambito de aplicación 
Art. 1 – Sujetos comprendidos*. Quedan comprendidas en esta ley y en sus 
normas reglamentarias las personas o entidades privadas o públicas –oficiales o 
mixtas– de la Nación, de las provincias o municipalidades que realicen 
intermediación habitual entre la oferta y la demanda de recur
Índice construido.


## Consultá en lenguaje natural lo que quieras sobre tu PDF

In [ ]:
# --- CELDA 8: Chat de preguntas y respuestas sobre el PDF ---

chat_engine = index.as_chat_engine(chat_mode="context", verbose=True)

markdown("## Consultá en lenguaje natural lo que quieras sobre tu PDF\n(Ejemplos reales en español)")

pregunta = "¿En qué casos puede el Banco Central revocar la autorización a una entidad financiera?"
respuesta = chat_engine.chat(pregunta)
markdown(f"**Pregunta:** {pregunta}\n\n**Respuesta:** {respuesta.response}")

# Proba con más preguntas
otras = [
    "¿Cuál es el objetivo principal de la Ley de Entidades Financieras?",
    "¿Qué requisitos se mencionan para la constitución de un banco en Argentina?",
    "Resumí el proceso de liquidación de una entidad financiera.",
    "¿La ley menciona la figura de cooperativas? ¿Qué dice al respecto?",
]

for pregunta in otras:
    r = chat_engine.chat(pregunta)
    markdown(f"**Pregunta:** {pregunta}\n\n**Respuesta:** {r.response}")

## Consultá en lenguaje natural lo que quieras sobre tu PDF
(Ejemplos reales en español)

**Pregunta:** ¿En qué casos puede el Banco Central revocar la autorización a una entidad financiera?

**Respuesta:** Según el texto, el Banco Central puede revocar la autorización a una entidad financiera en los siguientes casos:

* Revocar la aprobación para que todos o algunos accionistas de una entidad financiera continúen como tales, otorgando un plazo para la transferencia de dichas acciones, que no podrá ser inferior a diez días (inciso c) del artículo I).

No se menciona explícitamente otros casos en los que el Banco Central pueda revocar la autorización a una entidad financiera en el texto proporcionado.

**Pregunta:** ¿Cuál es el objetivo principal de la Ley de Entidades Financieras?

**Respuesta:** Según el texto, el objetivo principal de la Ley de Entidades Financieras es regular y controlar las entidades financieras, estableciendo normas y regulaciones para su funcionamiento y fiscalización, con el fin de proteger a los depositantes y a la economía en general.

**Pregunta:** ¿Qué requisitos se mencionan para la constitución de un banco en Argentina?

**Respuesta:** No se mencionan requisitos específicos para la constitución de un banco en Argentina en el texto proporcionado. Sin embargo, se establecen normas y regulaciones para el funcionamiento y fiscalización de las entidades financieras, incluyendo bancos, y se mencionan las situaciones en las que el Banco Central de la República Argentina puede revocar la autorización para que una entidad financiera funcione.

**Pregunta:** Resumí el proceso de liquidación de una entidad financiera.

**Respuesta:** Según el texto, el proceso de liquidación de una entidad financiera se desarrolla de la siguiente manera:

1. La autoridad competente (Banco Central de la República Argentina) resuelve la revocación de la autorización para que la entidad financiera funcione.
2. El liquidador judicial es designado y toma posesión de los bienes y activos de la entidad.
3. El liquidador presenta un informe al juez sobre el patrimonio de la entidad financiera y solicita la declaración de quiebra si advierte la cesación de pagos.
4. El juez dispone la liquidación judicial y ordena la realización de los bienes y activos de la entidad.
5. Los acreedores laborales y depositantes tienen prioridad en el pago de sus créditos.
6. El liquidador judicial puede solicitar la orden de allanamiento y el auxilio de la fuerza pública para asegurar el cumplimiento de la decisión del juez.
7. El liquidador judicial presenta un informe final al juez sobre la liquidación de la entidad financiera y el pago de los acreedores.

Es importante destacar que el proceso de liquidación judicial se rige por las normas establecidas en la Ley de Entidades Financieras y en el Código de Comercio.

**Pregunta:** ¿La ley menciona la figura de cooperativas? ¿Qué dice al respecto?

**Respuesta:** Sí, la ley menciona la figura de cooperativas. En el artículo 18, inciso a), se establece que las cajas de crédito cooperativas deberán ajustarse a ciertos requisitos, entre ellos:

* Las operaciones activas se realizarán preferentemente con asociados que se encuentren radicados o realicen su actividad económica en la zona de actuación en la que se le autorice a operar.
* Deberán remitir información periódica a sus asociados sobre su estado de situación patrimonial y capacidad de cumplimiento de las obligaciones adquiridas.

También se establece que las cajas de crédito cooperativas deberán asociarse en una cooperativa de grado superior especializada con capacidad, a satisfacción del Banco Central de la República Argentina.

En general, la ley establece regulaciones específicas para las cajas de crédito cooperativas, con el fin de proteger a los asociados y garantizar la estabilidad financiera de estas entidades.

### 🎓 Ejercicio para los alumnos

1. En la celda anterior modificá la variable `pregunta` y hacé tus propias consultas (probá tanto conceptos generales como citas textuales).
2. Cambiá el PDF por otro (manual universitario, otro texto legal, paper científico, etc.) y volvete a correr las celdas desde la descarga.
3. Preguntá por conceptos ambiguos: ¿cómo responde el modelo si el dato no está, o si la pregunta es imprecisa?
4. Observá el chat completo: ¿puede el LLM citar artículos específicos, fechas, nombres o hacer resúmenes?
5. ¿Qué mejoras imaginarías para un sistema así en tu carrera o trabajo?


Interactuá directamente con tu PDF. Escribí 'salir' para terminar.

In [ ]:
# --- CELDA 10: Chat REPL (opcional: diálogo interactivo) ---
while True:
    pregunta = input("Tu pregunta: ")
    if pregunta.lower() in ["salir", "exit", "quit"]:
        break
    resp = chat_engine.chat(pregunta)
    print("\nRespuesta:\n", resp.response)
    print("\n"+"-"*60)


Respuesta:
 El artículo 48 de la Ley de Entidades Financieras se refiere a la liquidación judicial de una entidad financiera. En este artículo se establece que el liquidador judicial deberá ser designado por el juez competente, según lo dispuesto por la Ley de Concursos y Quiebras.

------------------------------------------------------------
